# import各類module  

In [1]:
import pandas as pd

In [ ]:
import xgboost as xgb 
from xgboost import plot_importance #險是重要性
import pickle  #存MODEL用

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

# 自寫function

In [ ]:
#補中位數
def fillMedian(rawData,medianValue):
    for i in medianValue:
        med = rawData[i].median()
        rawData[i] = rawData[i].fillna(med)
    return rawData

#補平均值
def fillMean(rawData,medianValue):
    for i in medianValue:
        mean = rawData[i].mean()
        rawData[i] = rawData[i].fillna(mean)
    return rawData

#補non
def fillNon(rawData,nonValue):
    for i in nonValue:
        rawData[i] = rawData[i].fillna('none')
    return rawData

#補-1
def fillm1(rawData,nonValue):
    for i in nonValue:
        rawData[i] = rawData[i].fillna(-1)
    return rawData

#補0
def fillzero(rawData,nonValue):
    for i in nonValue:
        rawData[i] = rawData[i].fillna(0)
    return rawData

#補N
def fillN(rawData,NValue):
    for i in NValue:
        rawData[i] = rawData[i].fillna('N')
    return rawData

#One-Hot-Code
def oneHot(rawData,dummyValue):
    dummyData = pd.get_dummies(data = rawData, columns=dummyValue)
    return dummyData

# 資料檢查

In [ ]:
# test = pd.read_csv('D:\Python\dataset\TBN_CIF.csv') 
test[test['CUST_NO']=='5TLDP-9F8QUDYV3C']

# 為了用mssql整理，先把資料去除雙引號 

In [ ]:
# 去除資料雙引號
clean = pd.read_csv('D:\Python\dataset\TBN_CUST_BEHAVIOR.csv')
clean.to_csv('D:\Python\dataset\TBN_CUST_BEHAVIOR_clear.csv',index=False,sep='|')

# 瀏覽資料整理

In [ ]:
webView = pd.read_csv('D:\Python\dataset\TBN_CUST_BEHAVIOR.csv')

In [ ]:
webView[:3]

In [ ]:
new = webView['PAGE'].str.split('/',n=1, expand = True) 

In [ ]:
webView_sep = pd.merge(webView,new,how='left',left_index=True, right_index=True)

In [ ]:
webView_sep[:3]

In [ ]:
webView_sep = pd.merge(webView_sep,pd.DataFrame(le.fit_transform(webView_sep[1])),how='left',left_index=True, right_index=True)

In [ ]:
webView_sep = webView_sep.rename(columns = {'0_y':'siteLable'})[['CUST_NO','VISITDATE','siteLable']]

In [ ]:
webView_sep[:3]

In [ ]:
webView_sep_top_55 = webView_sep[webView_sep['siteLable'].isin([228,227,78,198,80,434,91,648,514,13,209,92,201,97,652,349,498,102,640,48,650,226,98,233,60,170,182,538,275,601,216,69,399,558,146,491,100,431,461,199,465,77,255,353,225,50,262,470,488,177,459,190,36,230,460])]

In [ ]:
webView_sep_top_55[:3]

In [ ]:
webView_sep_top_55_pivot = webView_sep_top_55.groupby(['CUST_NO','siteLable'], as_index=False).count().pivot(index='CUST_NO', columns='siteLable', values='VISITDATE').reset_index().fillna(0)

In [ ]:
webView_sep_top_55_pivot[:3]

In [ ]:
webView_sep_top_55_pivot[webView_sep_top_55_pivot['CUST_NO']=='AZTHNWQ_LXMGIMYG']

# 資料處理 (一次就好_cleanData.csv，處理完再用MSSQL處理)

In [ ]:
baseSet = pd.read_csv('D:\Python\dataset\CleanData.txt')

In [ ]:
baseSet[:2]

In [ ]:
# 整理補值欄位
fill_m1 = list(['CHILDREN_CNT','EDU_CODE','INCOME_RANGE_CODE','WORK_MTHS'])
fill_N = list(['GENDER_CODE'])
dummyVal = list(['CHILDREN_CNT','EDU_CODE','GENDER_CODE','INCOME_RANGE_CODE','WORK_MTHS'])

In [ ]:
# 開始補值
baseSet = fillN(baseSet,fill_N)
baseSet = fillm1(baseSet,fill_m1)
baseSet = oneHot(baseSet,dummyVal)

In [ ]:
# 輸出資料
baseSet.to_csv('D:\Python\dataset\cleanData.csv',index=False)

# 建模前資料整理 (將前階端資料用MSSQL整理後使用)

In [ ]:
# MSSQL處理完的BASE資料
cleanSet = pd.read_csv('D:\Python\dataset\data_v1.csv')
cc_set = pd.read_csv('D:\Python\dataset\data_v1_cc.csv')
fx_set = pd.read_csv('D:\Python\dataset\data_v1_fx.csv')
ln_set = pd.read_csv('D:\Python\dataset\data_v1_ln.csv')
wm_set = pd.read_csv('D:\Python\dataset\data_v1_wm.csv')

In [ ]:
cleanSet=pd.merge(cleanSet,webView_sep_top_55_pivot,how='left',left_on='cust_no', right_on='CUST_NO').fillna(0)
cc_set=pd.merge(cc_set,webView_sep_top_55_pivot,how='left',left_on='cust_no', right_on='CUST_NO').fillna(0)
fx_set=pd.merge(fx_set,webView_sep_top_55_pivot,how='left',left_on='cust_no', right_on='CUST_NO').fillna(0)
ln_set=pd.merge(ln_set,webView_sep_top_55_pivot,how='left',left_on='cust_no', right_on='CUST_NO').fillna(0)
wm_set=pd.merge(wm_set,webView_sep_top_55_pivot,how='left',left_on='cust_no', right_on='CUST_NO').fillna(0)

In [ ]:
list(cleanSet)

In [ ]:
orig_data = cleanSet[['AGE','CUST_START_DT','CHILDREN_CNT_-1 0','CHILDREN_CNT_0 0','CHILDREN_CNT_1 0','CHILDREN_CNT_2 0','CHILDREN_CNT_3 0','CHILDREN_CNT_4 0','CHILDREN_CNT_5 0','CHILDREN_CNT_6 0','CHILDREN_CNT_9 0','CHILDREN_CNT_10 0','EDU_CODE_-1 0','EDU_CODE_1 0','EDU_CODE_2 0','EDU_CODE_3 0','EDU_CODE_4 0','EDU_CODE_5 0','EDU_CODE_6 0','GENDER_CODE_F','GENDER_CODE_M','GENDER_CODE_N','INCOME_RANGE_CODE_-1 0','INCOME_RANGE_CODE_1 0','INCOME_RANGE_CODE_2 0','INCOME_RANGE_CODE_3 0','INCOME_RANGE_CODE_4 0','WORK_MTHS_-1 0','WORK_MTHS_1 0','WORK_MTHS_2 0','WORK_MTHS_3 0','WORK_MTHS_4 0','WORK_MTHS_5 0','start_CC','end_1_CC','end_2_CC','end_3_CC','end_4_CC','end_5_CC','start_fx','end_1_fx','end_2_fx','end_3_fx','end_4_fx','end_5_fx','start_ln','end_1_ln','end_2_ln','end_3_ln','end_4_ln','end_5_ln','start_wm','end_1_wm','end_2_wm','end_3_wm','end_4_wm','end_5_wm',13,36,48,50,60,69,77,78,80,91,92,97,98,100,102,146,170,177,182,190,198,199,201,209,216,225,226,227,228,230,233,255,262,275,349,353,399,431,434,459,460,461,465,470,488,491,498,514,538,558,601,640,648,650,652]].values
cc_data = cc_set[['AGE','CUST_START_DT','CHILDREN_CNT_-1 0','CHILDREN_CNT_0 0','CHILDREN_CNT_1 0','CHILDREN_CNT_2 0','CHILDREN_CNT_3 0','CHILDREN_CNT_4 0','CHILDREN_CNT_5 0','CHILDREN_CNT_6 0','CHILDREN_CNT_9 0','CHILDREN_CNT_10 0','EDU_CODE_-1 0','EDU_CODE_1 0','EDU_CODE_2 0','EDU_CODE_3 0','EDU_CODE_4 0','EDU_CODE_5 0','EDU_CODE_6 0','GENDER_CODE_F','GENDER_CODE_M','GENDER_CODE_N','INCOME_RANGE_CODE_-1 0','INCOME_RANGE_CODE_1 0','INCOME_RANGE_CODE_2 0','INCOME_RANGE_CODE_3 0','INCOME_RANGE_CODE_4 0','WORK_MTHS_-1 0','WORK_MTHS_1 0','WORK_MTHS_2 0','WORK_MTHS_3 0','WORK_MTHS_4 0','WORK_MTHS_5 0','start_CC','end_1_CC','end_2_CC','end_3_CC','end_4_CC','end_5_CC','start_fx','end_1_fx','end_2_fx','end_3_fx','end_4_fx','end_5_fx','start_ln','end_1_ln','end_2_ln','end_3_ln','end_4_ln','end_5_ln','start_wm','end_1_wm','end_2_wm','end_3_wm','end_4_wm','end_5_wm',13,36,48,50,60,69,77,78,80,91,92,97,98,100,102,146,170,177,182,190,198,199,201,209,216,225,226,227,228,230,233,255,262,275,349,353,399,431,434,459,460,461,465,470,488,491,498,514,538,558,601,640,648,650,652]].values
fx_data = fx_set[['AGE','CUST_START_DT','CHILDREN_CNT_-1 0','CHILDREN_CNT_0 0','CHILDREN_CNT_1 0','CHILDREN_CNT_2 0','CHILDREN_CNT_3 0','CHILDREN_CNT_4 0','CHILDREN_CNT_5 0','CHILDREN_CNT_6 0','CHILDREN_CNT_9 0','CHILDREN_CNT_10 0','EDU_CODE_-1 0','EDU_CODE_1 0','EDU_CODE_2 0','EDU_CODE_3 0','EDU_CODE_4 0','EDU_CODE_5 0','EDU_CODE_6 0','GENDER_CODE_F','GENDER_CODE_M','GENDER_CODE_N','INCOME_RANGE_CODE_-1 0','INCOME_RANGE_CODE_1 0','INCOME_RANGE_CODE_2 0','INCOME_RANGE_CODE_3 0','INCOME_RANGE_CODE_4 0','WORK_MTHS_-1 0','WORK_MTHS_1 0','WORK_MTHS_2 0','WORK_MTHS_3 0','WORK_MTHS_4 0','WORK_MTHS_5 0','start_CC','end_1_CC','end_2_CC','end_3_CC','end_4_CC','end_5_CC','start_fx','end_1_fx','end_2_fx','end_3_fx','end_4_fx','end_5_fx','start_ln','end_1_ln','end_2_ln','end_3_ln','end_4_ln','end_5_ln','start_wm','end_1_wm','end_2_wm','end_3_wm','end_4_wm','end_5_wm',13,36,48,50,60,69,77,78,80,91,92,97,98,100,102,146,170,177,182,190,198,199,201,209,216,225,226,227,228,230,233,255,262,275,349,353,399,431,434,459,460,461,465,470,488,491,498,514,538,558,601,640,648,650,652]].values
ln_data = ln_set[['AGE','CUST_START_DT','CHILDREN_CNT_-1 0','CHILDREN_CNT_0 0','CHILDREN_CNT_1 0','CHILDREN_CNT_2 0','CHILDREN_CNT_3 0','CHILDREN_CNT_4 0','CHILDREN_CNT_5 0','CHILDREN_CNT_6 0','CHILDREN_CNT_9 0','CHILDREN_CNT_10 0','EDU_CODE_-1 0','EDU_CODE_1 0','EDU_CODE_2 0','EDU_CODE_3 0','EDU_CODE_4 0','EDU_CODE_5 0','EDU_CODE_6 0','GENDER_CODE_F','GENDER_CODE_M','GENDER_CODE_N','INCOME_RANGE_CODE_-1 0','INCOME_RANGE_CODE_1 0','INCOME_RANGE_CODE_2 0','INCOME_RANGE_CODE_3 0','INCOME_RANGE_CODE_4 0','WORK_MTHS_-1 0','WORK_MTHS_1 0','WORK_MTHS_2 0','WORK_MTHS_3 0','WORK_MTHS_4 0','WORK_MTHS_5 0','start_CC','end_1_CC','end_2_CC','end_3_CC','end_4_CC','end_5_CC','start_fx','end_1_fx','end_2_fx','end_3_fx','end_4_fx','end_5_fx','start_ln','end_1_ln','end_2_ln','end_3_ln','end_4_ln','end_5_ln','start_wm','end_1_wm','end_2_wm','end_3_wm','end_4_wm','end_5_wm',13,36,48,50,60,69,77,78,80,91,92,97,98,100,102,146,170,177,182,190,198,199,201,209,216,225,226,227,228,230,233,255,262,275,349,353,399,431,434,459,460,461,465,470,488,491,498,514,538,558,601,640,648,650,652]].values
wm_data = wm_set[['AGE','CUST_START_DT','CHILDREN_CNT_-1 0','CHILDREN_CNT_0 0','CHILDREN_CNT_1 0','CHILDREN_CNT_2 0','CHILDREN_CNT_3 0','CHILDREN_CNT_4 0','CHILDREN_CNT_5 0','CHILDREN_CNT_6 0','CHILDREN_CNT_9 0','CHILDREN_CNT_10 0','EDU_CODE_-1 0','EDU_CODE_1 0','EDU_CODE_2 0','EDU_CODE_3 0','EDU_CODE_4 0','EDU_CODE_5 0','EDU_CODE_6 0','GENDER_CODE_F','GENDER_CODE_M','GENDER_CODE_N','INCOME_RANGE_CODE_-1 0','INCOME_RANGE_CODE_1 0','INCOME_RANGE_CODE_2 0','INCOME_RANGE_CODE_3 0','INCOME_RANGE_CODE_4 0','WORK_MTHS_-1 0','WORK_MTHS_1 0','WORK_MTHS_2 0','WORK_MTHS_3 0','WORK_MTHS_4 0','WORK_MTHS_5 0','start_CC','end_1_CC','end_2_CC','end_3_CC','end_4_CC','end_5_CC','start_fx','end_1_fx','end_2_fx','end_3_fx','end_4_fx','end_5_fx','start_ln','end_1_ln','end_2_ln','end_3_ln','end_4_ln','end_5_ln','start_wm','end_1_wm','end_2_wm','end_3_wm','end_4_wm','end_5_wm',13,36,48,50,60,69,77,78,80,91,92,97,98,100,102,146,170,177,182,190,198,199,201,209,216,225,226,227,228,230,233,255,262,275,349,353,399,431,434,459,460,461,465,470,488,491,498,514,538,558,601,640,648,650,652]].values
orig_data_CustNO = cleanSet['cust_no']
cc_target = cc_set['flag_cc']
fx_target = fx_set['flag_fx']
ln_target = ln_set['flag_ln']
wm_target = wm_set['flag_wm']

In [ ]:
origSet = xgb.DMatrix(orig_data)
cc_train = xgb.DMatrix(cc_data, cc_target)
fx_train = xgb.DMatrix(fx_data, fx_target)
ln_train = xgb.DMatrix(ln_data, ln_target)
wm_train = xgb.DMatrix(wm_data, wm_target)

# 開始建模

In [182]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 10,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
plst = params.items()

In [226]:
params_cc = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',  # 二元分類的问题
#     'objective': 'multi:softmax',  # 多分类的问题
#     'num_class': 10,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
plst_cc = params_cc.items()

In [227]:
params_fx = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',  # 二元分類的问题
#     'objective': 'multi:softmax',  # 多分类的问题
#     'num_class': 10,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
plst_fx = params_fx.items()

In [228]:
params_ln = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',  # 二元分類的问题
#     'objective': 'multi:softmax',  # 多分类的问题
#     'num_class': 10,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
plst_ln = params_ln.items()

In [229]:
params_wm = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',  # 二元分類的问题
#     'objective': 'multi:softmax',  # 多分类的问题
#     'num_class': 10,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 0,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}
plst_wm = params_wm.items()

In [ ]:
num_rounds = 500

In [230]:
cc_model = xgb.train(plst_cc, cc_train, num_rounds)

In [232]:
fx_model = xgb.train(plst_fx, fx_train, num_rounds)

In [183]:
ln_model = xgb.train(plst_ln, ln_train, num_rounds)

In [184]:
wm_model = xgb.train(plst_wm, wm_train, num_rounds)

# 重要屬性導出 

In [238]:
# dump model
cc_model.dump_model('D:\Python\dataset\dump.raw.txt')
# dump model with feature map
# cc_model.dump_model('dump.raw.txt','featmap.txt')

# 模型儲存與讀取 _ 一般

In [ ]:
cc_model.save_model('D:\Python\dataset\model_cc.pickle_20190224_01.model')
fx_model.save_model('D:\Python\dataset\model_fx.pickle_20190224_01.model')
ln_model.save_model('D:\Python\dataset\model_ln.pickle_20190224_01.model')
wm_model.save_model('D:\Python\dataset\model_wm.pickle_20190224_01.model')

# 模型儲存與讀取 _ pickle

In [ ]:
# save model _20190221
pickle.dump(cc_model, open("D:\Python\dataset\model_cc.pickle_20190224_01.dat", "wb"))
pickle.dump(fx_model, open("D:\Python\dataset\model_fx.pickle_20190224_01.dat", "wb"))
pickle.dump(ln_model, open("D:\Python\dataset\model_ln.pickle_20190224_01.dat", "wb"))
pickle.dump(wm_model, open("D:\Python\dataset\model_wm.pickle_20190224_01.dat", "wb"))

In [ ]:
#load model
# loaded_model = pickle.load(open("D:\Python\dataset\cc_model.pickle.dat", "rb"))

#  開始預測

In [218]:
# 預測
# ans_cc = cc_model.predict(origSet)
# ans_fx = fx_model.predict(origSet)
ans_ln = ln_model.predict(origSet)
ans_wm = wm_model.predict(origSet)

In [219]:
ans_ln = (ans_ln >= 0.85)*1
ans_wm = (ans_wm >= 0.85)*1

# 結果處理與輸出

In [220]:
# 合併結果
Output = pd.DataFrame({'CUST_NO':orig_data_CustNO, 'CC_IND':ans_cc.astype('int'),'FX_IND':ans_fx.astype('int'),'LN_IND':ans_ln.astype('int'),'WM_IND':ans_wm.astype('int')})
Output = Output[['CUST_NO','CC_IND','FX_IND','LN_IND','WM_IND']]

In [221]:
#只選主辦單位要求的名單
orig = pd.read_csv('D:\Python\dataset\TBN_Y_ZERO.csv')
orig_cust = pd.DataFrame(orig['CUST_NO'])
result = pd.merge(orig_cust, Output,how='left' ,on='CUST_NO')

#補0
fill_zero = list(['CC_IND','FX_IND','LN_IND','WM_IND'])
result = fillzero(result,fill_zero)

result.CC_IND = result.CC_IND.astype('int')
result.FX_IND = result.FX_IND.astype('int')
result.LN_IND = result.LN_IND.astype('int')
result.WM_IND = result.WM_IND.astype('int')

In [225]:
result.to_csv('D:\Python\dataset\output_20190224_03.csv',index=False,quoting=2)
result[:2]

,CUST_NO,CC_IND,FX_IND,LN_IND,WM_IND
0,_PT5HFBEZJKOZ934,0,0,0,0
1,6STXUMWZRDCGSDDU,1,0,0,0


In [ ]:
result[:10]

In [208]:
result[['CUST_NO','CC_IND']].groupby('CC_IND').count()

,CUST_NO
CC_IND,
0,26731
1,3269


In [209]:
result[['CUST_NO','FX_IND']].groupby('FX_IND').count()

,CUST_NO
FX_IND,
0,23149
1,6851


In [222]:
result[['CUST_NO','LN_IND']].groupby('LN_IND').count()

,CUST_NO
LN_IND,
0,29385
1,615


In [223]:
result[['CUST_NO','WM_IND']].groupby('WM_IND').count()

,CUST_NO
WM_IND,
0,28835
1,1165


# 模型效果

In [ ]:
# 準確綠  (此處沒做測試集)
cnt1 = 0
cnt2 = 0
for i in range(len(y_test)):
    if ans[i] == y_test[i]:
        cnt1 += 1
    else:
        cnt2 += 1

print("Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))

In [ ]:
# 重要變數
plot_importance(cc_model)
plt.show()

In [2]:
test = pd.read_csv('D:\Python\dataset\output_20190224_03.csv')

In [9]:
test.to_csv('D:\Python\dataset\output_20190226_test.csv',index=False,quoting=2)